# InputScope Analysis

This notebook provides a means for analysing data generated using the [InputScope](https://github.com/suurjaak/InputScope) mouse and keyboard interaction logger. Data from the tool is stored in an SQLite database. This notebook provides the ability to combine outputs from one or multiple InputScope databases if required.

#### Import Packages

In [ ]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import os

#### Set paths to InputScope databases, screen images and analysis outputs

In [ ]:
db_filepath = "./InputScope/"
db_filename = "inputscope.db"
screen_content = "./ScreenContent/"
outputs = "./Outputs/"

#### InputScope Sessions

InputScope 1.5 has introduced the concept of numbered 'Sessions' with a distinct start and end time. These weren't captured in the InputScope 1.4. Session times have been added as a CSV matching the format of the SQLite tables in the latest version of InputScope.

In [ ]:
sessions_df = pd.read_csv(db_filepath + 'sessions.csv', index_col=0)
sessions_df

#### Plot Count of Participants

In [ ]:
# Select required data
structural = sessions_df.loc[sessions_df["name"] == "Structural"]["participants"].tolist()
fitout = sessions_df.loc[sessions_df["name"] == "Fitout"]["participants"].tolist()
dates = sessions_df.loc[sessions_df["name"] == "Structural"]["day1"].tolist()
participants_df = pd.DataFrame({"Structural": structural, "Fitout": fitout}, index = dates)
# Plot data
ax = participants_df.plot.bar(figsize=(10,6), rot = 0, title = "Number of Participants in Weekly 3WLA Meetings")
# Add annotations to barsbars
for p in ax.patches:
    ax.annotate(str(p.get_height()).zfill(2), (p.get_x() + 0.05, p.get_height() + 0.2))
# Save figure as image

#### Save Plot of Participants Count

In [ ]:
fig = ax.get_figure()
fig.savefig(outputs + "Session_Participants.jpg")

#### Calculate Length of Sessions (mins)

In [ ]:
sessions_df["duration"] = ((sessions_df["end"] - sessions_df["start"]) / 60).astype(int)
sessions_df

#### Plot Length of Sessions

In [ ]:
# Select required data
structural = sessions_df.loc[sessions_df["name"] == "Structural"]["duration"].tolist()
fitout = sessions_df.loc[sessions_df["name"] == "Fitout"]["duration"].tolist()
dates = sessions_df.loc[sessions_df["name"] == "Structural"]["day1"].tolist()
participants_df = pd.DataFrame({"Structural": structural, "Fitout": fitout}, index = dates)
# Plot data
ax = participants_df.plot.bar(figsize=(10,6), rot = 0, title = "Duration of Weekly 3WLA Meetings (mins)")
# Add annotations to barsbars
for p in ax.patches:
    ax.annotate(str(p.get_height()).zfill(2), (p.get_x() + 0.05, p.get_height() + 0.8))

#### Save Plot of Length of Sessions

In [ ]:
# Save figure as image
fig = ax.get_figure()
fig.savefig(outputs + "Session_Durations.jpg")

#### Define Function to Create the Database Connection and Extract Data

If you try to connect to an SQLite database file that does not exist, SQLite will automatically create the new database for you. However, any folder's specified in the filepath to the database must exist before you execute the program.

In [ ]:
# Define the function to connect to an SQLite database
def create_connection(db_file):
    
    # Create a new database connection object
    db = None
    try:
        # Try the database connection
        db = sqlite3.connect(db_file)
        
        # Query tables and store in dataframes
        app_events = pd.read_sql_query("SELECT * FROM app_events", db)
        counts = pd.read_sql_query("SELECT * FROM counts", db)
        screen_sizes = pd.read_sql_query("SELECT * FROM screen_sizes", db)
        clicks = pd.read_sql_query("SELECT * FROM clicks", db)
        moves = pd.read_sql_query("SELECT * FROM moves", db)
        scrolls = pd.read_sql_query("SELECT * FROM scrolls", db)
        keys = pd.read_sql_query("SELECT * FROM keys", db)
        combos = pd.read_sql_query("SELECT * FROM combos", db)
        
        # Create dictionary of datatables
        datatables = {"app_events": app_events,
                      "counts": counts,
                      "screen_sizes": screen_sizes,
                      "clicks": clicks,
                      "moves": moves,
                      "scrolls": scrolls,
                      "keys": keys,
                      "combos": combos}
        
        # Return the datatables
        return datatables
        
    # Catch and print any errors
    except Error as e:
        print(e)
    # Close the connection after execution of the try/except block
    finally:
        if db:
            db.close()

#### Extract Contents for the InputScope Database

In [ ]:
# Loop through database folder
for filename in os.listdir(db_filepath):
    print("Read database - " + filename)
    db_tables = create_connection(db_filepath + db_filename)

#### Display Counts of Interactions

In [ ]:
db_tables["counts"]

#### Plot Comparison of Interaction Type Counts by Date

In [ ]:
# Select required data
moves = db_tables["counts"].loc[db_tables["counts"]["type"] == "moves"]["count"].tolist()
clicks = db_tables["counts"].loc[db_tables["counts"]["type"] == "clicks"]["count"].tolist()
scrolls = db_tables["counts"].loc[db_tables["counts"]["type"] == "scrolls"]["count"].tolist()
keys = db_tables["counts"].loc[db_tables["counts"]["type"] == "keys"]["count"].tolist()
combos = db_tables["counts"].loc[db_tables["counts"]["type"] == "combos"]["count"].tolist()
dates = db_tables["counts"].loc[db_tables["counts"]["type"] == "clicks"]["day"].tolist()

counts_df = pd.DataFrame({"Clicks": clicks, "Scrolls" : scrolls, "Keys" : keys, "Combos" : combos}, index = dates)
subplot_df = pd.DataFrame({"Mouse Moves": moves}, index = dates)
# Plot data
ax1 = counts_df.plot.bar(figsize=(10,6), rot = 0, title = "Count of Screen Interactions in Weekly 3WLA Meetings by Type")
ax2 = subplot_df.plot.bar(figsize=(10,6), rot = 0, title = "Distance of Mouse Moves (screen pixels) in Weekly 3WLA Meetings")
# Add annotations to barsbars
for p in ax1.patches:
    ax1.annotate(str(p.get_height()).zfill(2), (p.get_x() + 0.00, p.get_height() + 2))
for p in ax2.patches:
    ax2.annotate(str(p.get_height()).zfill(2), (p.get_x() + 0.08, p.get_height() + 30))

#### Save Plots for Interactions

In [ ]:
# Save figures as images
fig1 = ax1.get_figure()
fig1.savefig(outputs + "Interactions_Count.jpg")
fig2 = ax2.get_figure()
fig2.savefig(outputs + "Mouse_Move_Distance.jpg")

#### Display Screen Sizes

In [ ]:
db_tables["screen_sizes"]

#### Plot Heatmap

Visualisation based on the following: https://stackoverflow.com/questions/36957149/density-map-heatmaps-in-matplotlib/36958298

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from scipy.stats.kde import gaussian_kde

x,y = db_tables["clicks"]["x"], db_tables["clicks"]["y"]

k = gaussian_kde(np.vstack([x,y]))
xi, yi = np.mgrid[x.min():x.max():x.size**0.5*1j,y.min():y.max():y.size**0.5*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))

# Custom colourmap can be used instead of Matplotlib defaults
colors = ['#ffffff', '#4eb3d3', '#a8ddb5', '#fff33b', '#fdc70c', '#f3903f', '#ed683c', '#e93e3a']
cm = ListedColormap(colors)

# Set figure size
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(title = "Heatmap of Click Interations (Screen 2)")

# cmap specifies the colourmap and alpha makes the plots semitransparent
ax.contourf(xi, yi, zi.reshape(xi.shape), cmap=cm, alpha=0.6)

ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())

# Hide axis tick labels
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

# Read image to overlay
img = plt.imread(screen_content + 'image.png')

# Add overlay to plot
ax.imshow(img, extent=[x.min(), x.max(), y.min(), y.max()], aspect='auto')

# Save plot
plt.savefig(outputs + 'clicks.jpg')